In [ ]:
# Author                 :  Mohammad Noor Ul Hasan
# Start Date             :  13/Feb/19

# Dependencies(Modules ) :  keras 
# 							pandas
# 							cv2
# 							numpy
# Last Edited            :  10/Apr/19


In [4]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

dataset_path = 'dataset/fer2013.csv'
image_shape = (48,48)
dataset_path

'dataset/fer2013.csv'

In [5]:
''' Load dataset, extract images and emotions '''

def load_dataset():
    # load csv
    data = pd.read_csv(dataset_path)
#     print('\nDataset glimpse : \n',data.head())
    
    # save all the pixels values to list 'pixels' 
    pixels_of_images = data['pixels'].tolist()
#     print('\nTotal number of images : ',len(pixels_of_images))

#     print('\nData type of pixel values : ',type(pixels_of_images[0]),'\nLength of pixels per image : ' ,len(pixels_of_images[0]))
    ## which is not equals to 2304
    
    # to store faces
    images = []
    # get all images with pixels values 48*48=2304
    for image_pixels in pixels_of_images:
        # face contains pixels value of single image
        image = [int(pixel) for pixel in image_pixels.split(' ')]
#         print('\n\n Image length (Single image pixels with 48*48 dimensions):', len(image))
        # convert image into array
        image = np.asarray(image).reshape(image_shape[0], image_shape[1])
        # save the image into images array
        images.append(image.astype('float32'))
        

    # convert images list into array
    images = np.asarray(images)
    print('\nShape before expanding dimensions : ',images.shape)
    
    # expand images dimensions
    images = np.expand_dims(images, -1)
    print('\nShape after expanding dimensions : ',images.shape)
    
    # extract emotions and do one-hot encoding
    emotions = pd.get_dummies(data['emotion'])
    
    return images, emotions


''' Preprocess all the images '''

def preprocess_images(images, temp=True):
    images = images.astype('float32')
    images = images/255.0
    
#     print(images[0][0])
    if temp :
        images = images - 0.5
        images = images * 2.0
        
#         print(images[0][0])
    return images


In [6]:
''' Driver Code '''

# load dataset 
images, emotions = load_dataset()

# preprocess the images
images = preprocess_images(images, True)



Shape before expanding dimensions :  (35887, 48, 48)

Shape after expanding dimensions :  (35887, 48, 48, 1)


In [ ]:
xtrain, xtest,ytrain,ytest = train_test_split(images, emotions,test_size=0.2,shuffle=True)

from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Activation, Convolution2D, Dropout, Conv2D, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Input, MaxPooling2D, SeparableConv2D
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np
 
# parameters
batch_size = 32
num_epochs = 110
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 50
base_path = 'models/'
l2_regularization=0.01
 
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
 
# model parameters
regularization = l2(l2_regularization)
 
# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
 
# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)
 
model = Model(img_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
 
# callbacks
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
 
model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 46, 46, 8)    72          input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_85 (BatchNo (None, 46, 46, 8)    32          conv2d_43[0][0]                  
__________________________________________________________________________________________________
activation_37 (Activation)      (None, 46, 46, 8)    0           batch_normalization_85[0][0]     
__________________________________________________________________________________________________
conv2d_44 

Epoch 1/110
898/897 [==============================] - 204s 228ms/step - loss: 1.7437 - acc: 0.3354 - val_loss: 1.5631 - val_acc: 0.4172

Epoch 00001: val_loss improved from inf to 1.56314, saving model to models/_mini_XCEPTION.01-0.42.hdf5
Epoch 2/110
898/897 [==============================] - 206s 230ms/step - loss: 1.4929 - acc: 0.4410 - val_loss: 1.4780 - val_acc: 0.4558

Epoch 00002: val_loss improved from 1.56314 to 1.47803, saving model to models/_mini_XCEPTION.02-0.46.hdf5
Epoch 3/110
898/897 [==============================] - 206s 229ms/step - loss: 1.3807 - acc: 0.4821 - val_loss: 1.4424 - val_acc: 0.4965

Epoch 00003: val_loss improved from 1.47803 to 1.44239, saving model to models/_mini_XCEPTION.03-0.50.hdf5
Epoch 4/110
898/897 [==============================] - 202s 225ms/step - loss: 1.3164 - acc: 0.5064 - val_loss: 1.2885 - val_acc: 0.5258

Epoch 00004: val_loss improved from 1.44239 to 1.28847, saving model to models/_mini_XCEPTION.04-0.53.hdf5
Epoch 5/110
898/897 [===